In [90]:
import pandas as pd
test_args_with_q_scores = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\raw\ArgKP_2021_shared_task\test_args_with_scores.csv', index_col=None)

In [91]:
test_args_with_q_scores.head(2)

,arg_id,argument,topic,stance,MACE-P,sents,sents_with_scores
0,arg_0_0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should be mandatory,-1,0.714195,"[""Routine child vaccinations isn't mandatory since children don't spread the virus""]","[(""Routine child vaccinations isn't mandatory since children don't spread the virus"", 0.714194655418396)]"
1,arg_0_1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,Routine child vaccinations should be mandatory,-1,0.871067,['Routine child vaccinations should not be mandatory because children may not bear the side effects of it.'],"[('Routine child vaccinations should not be mandatory because children may not bear the side effects of it.', 0.8710675239562988)]"


In [92]:
import ast

sentences = []
scores = []
topics = []
stances = []

for index, row in test_args_with_q_scores.iterrows():
    sents = ast.literal_eval(row["sents_with_scores"])
    for sent, score in sents:
        sentences.append(sent)
        scores.append(score)
        topics.append(row['topic'])
        stances.append(row['stance'])

all_pot_sent_df = pd.DataFrame({"topic": topics, "stance": stances, "sentence": sentences, "q_score": scores})


In [93]:
all_pot_sent_df['word_count'] = all_pot_sent_df['sentence'].str.split().str.len()

In [94]:
print(len(all_pot_sent_df))
all_pot_sent_df.head(2)

759


,topic,stance,sentence,q_score,word_count
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations isn't mandatory since children don't spread the virus,0.714195,11
1,Routine child vaccinations should be mandatory,-1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,0.871068,17


In [95]:
all_pot_sent_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_raw.csv', index=False)

In [87]:
##### get sentences with good quality
all_pot_sent_df = all_pot_sent_df[all_pot_sent_df['q_score'] > 0.5]
# get sentences with length between 3-36 tokens
mask = (all_pot_sent_df['word_count'] > 5) & (all_pot_sent_df['word_count'] < 20)
all_pot_sent_df = all_pot_sent_df.loc[mask]
print(len(all_pot_sent_df))

517


In [88]:
# drop syntatically similar sentences
all_pot_sent_df =  all_pot_sent_df.drop_duplicates(["topic", "stance", "sentence"])
print(len(all_pot_sent_df))

517


In [89]:
all_pot_sent_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_cleaned_basic_filtering.csv', index=False)

In [119]:
# to reduce similar potential sentences semantically, we match them to each other and remove similar sentences
# create a cross sentences df
unique_topics = all_pot_sent_df['topic'].unique()
unique_stances = [-1, 1]

arg1 = []
arg2 = []
topics = []
stances = []
q_score_1 = []
q_score_2 = []
word_count1 = []
word_count2 = []

for t in unique_topics:
    for s in unique_stances:
        df_f = all_pot_sent_df[all_pot_sent_df['topic'] == t]
        df_f = df_f[df_f['stance'] == s]
        unique_sents = df_f["sentence"].unique()
        for s1 in unique_sents:
            row_1 = df_f[df_f['sentence'] == s1].iloc[0]
            for s2 in unique_sents:
                row_2 = df_f[df_f['sentence'] == s2].iloc[0]
                arg1.append(s1)
                arg2.append(s2)
                topics.append(row_1['topic'])
                stances.append(row_1['stance'])
                q_score_1.append(row_1['q_score'])
                q_score_2.append(row_2['q_score'])
                word_count1.append(row_1['word_count'])
                word_count2.append(row_2['word_count'])

        
cross_df = pd.DataFrame({
    "sent1" : arg1 ,
    "sent2": arg2,
    "topic": topics,
    "stance": stances,
    "q_score_1": q_score_1,
    "q_score_2": q_score_2,
    "word_count1": word_count1,
    "word_count2": word_count2
})

In [113]:
cross_df.head(2)

,sent1,sent2,topic,stance,q_score_1,q_score_2,word_count1,word_count2
0,Routine child vaccinations isn't mandatory since children don't spread the virus <SEP> Routine child vaccinations should be mandatory,Routine child vaccinations isn't mandatory since children don't spread the virus <SEP> Routine child vaccinations should be mandatory,Routine child vaccinations should be mandatory,-1,0.714195,0.714195,11,11
1,Routine child vaccinations isn't mandatory since children don't spread the virus <SEP> Routine child vaccinations should be mandatory,Routine child vaccinations should not be mandatory because children may not bear the side effects of it. <SEP> Routine child vaccinations should be mandatory,Routine child vaccinations should be mandatory,-1,0.714195,0.871068,11,17


In [120]:
cross_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_cross_df.csv', index=False)

In [121]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

# Two lists of sentences
sentences1 = cross_df['sent1']
sentences2 =cross_df['sent2']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True, batch_size=8, show_progress_bar=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True, batch_size=8, show_progress_bar=True)

Batches:   0%|          | 0/6152 [00:00<?, ?it/s]

Batches:   0%|          | 0/6152 [00:00<?, ?it/s]

In [122]:
import torch


def cos_sim(a, b): 
    """ Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j. :return: Matrix with res[i][j]  = cos_sim(a[i], b[j]) """ 
    if not isinstance(a, torch.Tensor): 
        a = torch.tensor(a) 
    if not isinstance(b, torch.Tensor): 
        b = torch.tensor(b) 
    if len(a.shape) == 1: 
        a = a.unsqueeze(0) 
    if len(b.shape) == 1: 
        b = b.unsqueeze(0) 
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1) 
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1) 
    res = torch.mm(a_norm, b_norm.transpose(0, 1))
    return res
 

cosine_scores = cos_sim(embeddings1.cpu(), embeddings2.cpu())


In [123]:


sent1 = []
sent2 = []
scores = []

#Output the pairs with their score
for i in range(len(sentences1)):
    sent1.append(sentences1[i])
    sent2.append(sentences2[i])
    scores.append(cosine_scores[i][i].item())
    
res_df = pd.DataFrame({"sent1": sent1, "sent2": sent2, "score": scores})

In [133]:
res_df = res_df[res_df['score'] > 0.45]
res_df =  res_df[res_df['sent1'] != res_df['sent2']]

In [134]:
res_df.sort_values(by='score')

,sent1,sent2,score
15635,"To avoid spreading diseases in schools and day care centers, all children must be vaccinated.",It has to be mandatory to avoid more infections,0.450028
12687,It has to be mandatory to avoid more infections,"To avoid spreading diseases in schools and day care centers, all children must be vaccinated.",0.450028
43893,The USA has so many opportunities for everyone working their that it is a good country to live in.,if it is a country with good salaries,0.450062
47849,if it is a country with good salaries,The USA has so many opportunities for everyone working their that it is a good country to live in.,0.450062
10693,Children should be protected at all costs.,childhood vaccination is necessary as it helps the growth of children,0.450080
...,...,...,...
33967,Social media platforms should be regulated by the government to prevent any potential crimes,Social media platforms should be regulated by the government to prevent the crime,0.988243
18514,Vaccines should be a mandatory policy so that children do not suffer from these covid problems,Vaccines should be a mandatory policy so that children do not suffer from these covid problems in the future,0.994360
18648,Vaccines should be a mandatory policy so that children do not suffer from these covid problems in the future,Vaccines should be a mandatory policy so that children do not suffer from these covid problems,0.994360
7276,Routine childhood vaccinations should be mandatory for virus prevention,Routine childhood vaccinations should be mandatory for virus prevention,1.000000


In [135]:
# add scores and word counts
res_df_enhanced = pd.merge(res_df, all_pot_sent_df, left_on="sent1", right_on="sentence", how="inner")
res_df_enhanced = res_df_enhanced.rename(columns={'q_score': 'q_score_1', 'word_count': 'word_count_1'})


In [136]:
res_df_enhanced = pd.merge(res_df_enhanced, all_pot_sent_df, left_on="sent2", right_on="sentence", how="inner")
res_df_enhanced = res_df_enhanced.rename(columns={'q_score': 'q_score_2', 'word_count': 'word_count_2'})

In [137]:
res_df_enhanced = res_df_enhanced[['sent1', 'sent2', 'score', 'topic_x', 'stance_x', 'q_score_1', 'word_count_1', 'q_score_2', 'word_count_2']]

In [138]:
res_df_enhanced = res_df_enhanced.rename(columns={'topic_x': 'topic', 'stance_x': 'stance'})

In [139]:
res_df_enhanced.head(2)

,sent1,sent2,score,topic,stance,q_score_1,word_count_1,q_score_2,word_count_2
0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,0.869770,Routine child vaccinations should be mandatory,-1,0.714195,11,0.871068,17
1,Routine child vaccinations should not be necessary as children can't catch the disease,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,0.895673,Routine child vaccinations should be mandatory,-1,0.820775,13,0.871068,17


In [140]:
res_df_enhanced.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_cross_with_sim_scores_df.csv', index=False)

In [1]:
import pandas as pd
res_df_enhanced = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_cross_with_sim_scores_df.csv', index_col=None)

In [2]:
# get count grouped by topic, stance , if tie break with q score
THRESHOLD = 0.45
match_count_df = res_df_enhanced[res_df_enhanced['score'] > THRESHOLD]
match_count_df =  match_count_df[match_count_df['sent1'] != match_count_df['sent2']]

In [3]:
only_counts_df = match_count_df.groupby(by='sent1').size().reset_index(name='counts')
match_count_df = pd.merge(match_count_df, only_counts_df, on='sent1')

In [4]:
match_count_df = match_count_df[['sent1', 'topic', 'stance', 'q_score_1', 'word_count_1', 'counts']].drop_duplicates()

In [5]:
len(match_count_df)

510

In [53]:
# Get top n matched sentences  from every topic/stance
N = 20
top_df = match_count_df.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by=["counts", "q_score_1", "word_count_1"], ascending=(False, False, True))).reset_index(drop=True)

In [54]:
len(top_df)

510

In [55]:
top_df.head(18)

,sent1,topic,stance,q_score_1,word_count_1,counts
0,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,0.862094,18,61
1,it is the duty of parents to choose whether or not to vaccinate their children,Routine child vaccinations should be mandatory,-1,0.842486,15,61
2,child vaccinations should not be mandatory to avoid any bad effects on them,Routine child vaccinations should be mandatory,-1,0.773095,13,61
3,child vaccinations shouldn't be mandatory because the vaccine can have side effects on children,Routine child vaccinations should be mandatory,-1,0.878956,14,60
4,Child vaccination should not be mandatory as there are often side effects from them,Routine child vaccinations should be mandatory,-1,0.842266,14,60
5,"It shouldn't be mandatory , every parent should be able to decide whether to vaccinate their childs or not.",Routine child vaccinations should be mandatory,-1,0.780102,19,60
6,Vaccines in children should not be mandatory because they can have consequences for their health in the future,Routine child vaccinations should be mandatory,-1,0.885895,18,59
7,vaccination of children should be limited as it affects child development,Routine child vaccinations should be mandatory,-1,0.858884,11,59
8,Child vaccination should not be mandatory because they don't suffer from symptoms,Routine child vaccinations should be mandatory,-1,0.752868,12,59
9,Child vaccination shouldn't be mandatory because is dangerous,Routine child vaccinations should be mandatory,-1,0.578729,8,59


In [73]:
# filter redundant sentences 

FILTER_THRESHOLD = 0.4

top_filtered_sents = set()

for index, row in top_df.iterrows():
    s1 = row['sent1']
    if len(top_filtered_sents) == 0:
        top_filtered_sents.add(s1)
        continue
    
    sim_sentences = res_df_enhanced[(res_df_enhanced.sent1 == s1)]
    sim_sentences = sim_sentences[(sim_sentences.sent2.isin(top_filtered_sents))]
    if len(sim_sentences) == 0:
        top_filtered_sents.add(s1)
        continue
    if max(sim_sentences['score']) < FILTER_THRESHOLD:
        top_filtered_sents.add(s1)
    

top_filtered_df = pd.DataFrame({"sent1": list(top_filtered_sents)})

In [74]:
top_filtered_df = pd.merge(top_filtered_df, top_df, on='sent1')

In [75]:
print(len(top_filtered_df))
top_filtered_df.head(2)

50


,sent1,topic,stance,q_score_1,word_count_1,counts
0,there is a very high unemployment rate and services are very expensive,The USA is a good country to live in,-1,0.846090,12,2
1,The state will then have the power to silence government critics.,Social media platforms should be regulated by the government,-1,0.803378,11,1


In [76]:
N = 20
top_filtered_df = top_filtered_df.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by=["counts", "q_score_1", "word_count_1"], ascending=(False, False, True)).head(N)).reset_index(drop=True)

In [77]:
top_filtered_df.topic.value_counts()

The USA is a good country to live in                            20
Social media platforms should be regulated by the government    16
Routine child vaccinations should be mandatory                  14
Name: topic, dtype: int64

In [80]:
top_filtered_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_cleaned.csv', index=False)

In [2]:
import pandas as pd
top_filtered_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_cleaned.csv', index_col=None)

In [5]:
top_filtered_df.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by=["counts", "q_score_1", "word_count_1"], ascending=(False, False, True)).head(4)).reset_index(drop=True)

,sent1,topic,stance,q_score_1,word_count_1,counts
0,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,0.862094,18,61
1,someone from the child population could suffer side effects,Routine child vaccinations should be mandatory,-1,0.837692,9,19
2,"They should not be mandatory, it is a personal decision",Routine child vaccinations should be mandatory,-1,0.717764,10,17
3,parents should have the freedom to decide what they consider best for their children,Routine child vaccinations should be mandatory,-1,0.906442,14,13
4,Child vaccinations should be mandatory to prevent children becoming seriously ill from diseases and spreading infections,Routine child vaccinations should be mandatory,1,0.930479,16,121
5,"Yes, it is necessary to preserve the life of the child",Routine child vaccinations should be mandatory,1,0.683039,11,27
6,"taking into account that the virus is very contagious, it should be mandatory to avoid future mutations",Routine child vaccinations should be mandatory,1,0.822368,17,20
7,"Yes, it is a way to guarantee health",Routine child vaccinations should be mandatory,1,0.637232,8,11
8,Government regulation of social media would be harmful to democracy,Social media platforms should be regulated by the government,-1,0.910105,10,57
9,they are a privates companies and shouldn't have to be regulated,Social media platforms should be regulated by the government,-1,0.757534,11,23


In [3]:
pd.set_option('display.max_colwidth', None)
top_filtered_df.head(40)

,sent1,topic,stance,q_score_1,word_count_1,counts
0,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,0.862094,18,61
1,someone from the child population could suffer side effects,Routine child vaccinations should be mandatory,-1,0.837692,9,19
2,"They should not be mandatory, it is a personal decision",Routine child vaccinations should be mandatory,-1,0.717764,10,17
3,parents should have the freedom to decide what they consider best for their children,Routine child vaccinations should be mandatory,-1,0.906442,14,13
4,could be damaged by side effects I honestly don't consider it safe,Routine child vaccinations should be mandatory,-1,0.643530,12,10
5,"No, the free will of the people should be respected",Routine child vaccinations should be mandatory,-1,0.611934,10,3
6,It goes against some religions that prohibit it.,Routine child vaccinations should be mandatory,-1,0.683977,8,1
7,Child vaccinations should be mandatory to prevent children becoming seriously ill from diseases and spreading infections,Routine child vaccinations should be mandatory,1,0.930479,16,121
8,"Yes, it is necessary to preserve the life of the child",Routine child vaccinations should be mandatory,1,0.683039,11,27
9,"taking into account that the virus is very contagious, it should be mandatory to avoid future mutations",Routine child vaccinations should be mandatory,1,0.822368,17,20


In [8]:
import pandas as pd
test_ds_labeled_counts = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_ds_labeled_counts.csv", index_col=None)

In [6]:
import rouge
# evaluate to ground truth using rouge
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           limit_length=True,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [11]:
# comparing test_ds_labeled_counts with counst_df
# for every topic, stance => compare predicted kp with every labeled kp and take max score
import numpy as np
from itertools import product
unique_topics = test_ds_labeled_counts['topic'].unique()
unique_stances = [-1, 1]

topics = []
stances = []
scores = []

for t in unique_topics:
    for s in unique_stances:
        test_df = test_ds_labeled_counts[test_ds_labeled_counts['topic'] == t]
        test_df = test_df[test_df['stance'] == s]
        res_df = top_filtered_df[top_filtered_df['topic'] == t]
        res_df = res_df[res_df['stance'] == s]
        
        unique_predicted_kps = res_df['sent1'].unique()
        unique_labeled_kps = test_df['key_point'].unique()
        kp_scores = []
        for p_kp, l_kp in product(unique_predicted_kps, unique_labeled_kps) :
                score = evaluator.get_scores(p_kp, l_kp)
                kp_scores.append(score['rouge-l']['f'])    

        topics.append(t)
        stances.append(s)
        scores.append(np.mean(kp_scores))
        
    
results_df = pd.DataFrame({"topic": topics, "stance": stances,  "r_scores": scores})
    

In [12]:
results_df.head(6)

,topic,stance,r_scores
0,Routine child vaccinations should be mandatory,-1,0.123258
1,Routine child vaccinations should be mandatory,1,0.160126
2,Social media platforms should be regulated by the government,-1,0.118588
3,Social media platforms should be regulated by the government,1,0.145223
4,The USA is a good country to live in,-1,0.199336
5,The USA is a good country to live in,1,0.128515
